# 优化超参数

有许多机器学习算法需要*超参数* （影响训练但无法通过训练数据本身确定的参数值）。例如，训练逻辑回归模型时，可以使用*正则化率*超参数来抵消模型中的偏差；或者训练卷积神经网络时，可以使用*学习速率*和*批大小等超*参数来分别控制权重的调整方式以及以小批量方式处理的数据项数量。超参数值的选择会显著影响已训练模型的性能或训练该模型所需的时间；通常需要尝试多种组合以找到最佳解决方案。

在本例中，你将使用两个超参数训练分类模型，但是原则适用于可使用 Azure 机器学习训练的任何类型的模型。

## 连接到工作区

首先，请连接到你的工作区。

> **备注**： 如果尚未与 Azure 订阅建立经过身份验证的会话，则系统将提示你通过执行以下操作进行身份验证：单击链接，输入验证码，然后登录到 Azure。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 从保存的配置文件加载工作区
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 准备数据

在本实验室中，你将使用包含糖尿病患者详细信息的数据集。运行以下单元格以创建此数据集（如果已有此数据集，则会采用现有数据集）

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # 将糖尿病 csv 文件上传到 /data 中
                        target_path='diabetes-data/', # 将其放在数据存储的文件夹路径中
                        overwrite=True, # 替换名称相同的现有文件
                        show_progress=True)

    #从数据存储上的路径创建表格数据集（这可能需要一些时间）
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # 注册表格数据集
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## 准备训练脚本

我们现在来为将用于训练模型的训练脚本创建一个文件夹。

In [ ]:
import os

experiment_folder = 'diabetes_training-hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

print('Folder ready.')

现在创建 Python 脚本以训练模型。在本例中，你将使用*梯度提升*算法训练分类模型。脚本必须包括：

- 你想要优化的每个超参数的参数（在本例中即梯度提升算法的学习速率和估算器数量）
- 用于记录要优化的性能指标的代码（在本例中将记录 AUC 和准确度，以便可以选择针对其中一项优化模型）

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 导入库
import argparse, joblib, os
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, roc_curve

# 获取试验运行上下文
run = Run.get_context()

# 获取脚本参数
parser = argparse.ArgumentParser()

# 输入数据集
parser.add_argument("--input-data", type=str, dest='input_data', help='training dataset')

# 超参数
parser.add_argument('--learning_rate', type=float, dest='learning_rate', default=0.1, help='learning rate')
parser.add_argument('--n_estimators', type=int, dest='n_estimators', default=100, help='number of estimators')

# 向 args 集合添加参数
args = parser.parse_args()

# 记录超参数值
run.log('learning_rate',  np.float(args.learning_rate))
run.log('n_estimators',  np.int(args.n_estimators))

# 加载糖尿病数据集
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe() # 从估算器输入中获取训练数据

# 分隔特征和标签
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 将数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 使用指定超参数训练梯度提升分类模型
print('Training a classification model')
model = GradientBoostingClassifier(learning_rate=args.learning_rate,
                                   n_estimators=args.n_estimators).fit(X_train, y_train)

# 计算准确度
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# 计算 AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# 保存运行输出中的模型
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 创建计算

超参数优化包括使用不同的超参数值运行多个训练迭代，并比较结果模型的性能指标。为了有效地做到这一点，我们将借助按需云计算并创建一个群集，这样即可同时运行多个训练迭代。

使用以下代码指定一个 Azure 机器学习计算群集（如果不存在，则会创建该群集）。

> **重要信息**： 在运行以下代码之前，请先将代码中的 *your-compute-cluster*  更改为你的计算群集的名称！群集名称必须是长度在 2 到 16 个字符之间的全局唯一名称。有效字符是字母、数字和 - 字符。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # 检查现有的计算目标
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # 如果尚不存在，请创建它
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

## 运行超参数优化试验

Azure 机器学习包括通过 *Hyperdrive* 试验提供的超参数优化功能。这些试验启动了多个子运行，每个子运行都有不同的超参数组合。可以确定生成最佳模型的运行（由要优化的记录目标性能指标确定），并选择其训练后的模型进行注册和部署。

> **备注**： 在本例中，我们没有采用早停法。只有当训练脚本执行多个训练迭代，记录每个迭代的主要指标时，早停法才有作用。这种方法通常用于多次迭代的深度神经网络模型训练。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails

# 创建用于试验的 Python 环境
sklearn_env = Environment("sklearn-env")

# 确保已安装所需的包（我们需要 scikit-learn、Azure ML 默认值和 Azure ML dataprep）
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-dataprep[pandas]'])
sklearn_env.python.conda_dependencies = packages

# 获取训练数据集
diabetes_ds = ws.datasets.get("diabetes dataset")

# 创建脚本配置
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                # Add non-hyperparameter arguments -in this case, the training dataset
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=sklearn_env,
                                compute_target = training_cluster)

# 对一系列参数值进行采样
params = GridParameterSampling(
    {
        # Hyperdrive will try 6 combinations, adding these as script arguments
        '--learning_rate': choice(0.01, 0.1, 1.0),
        '--n_estimators' : choice(10, 100)
    }
)

# 配置 hyperdrive 设置
hyperdrive = HyperDriveConfig(run_config=script_config, 
                          hyperparameter_sampling=params, 
                          policy=None, # No early stopping policy
                          primary_metric_name='AUC', # Find the highest AUC metric
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6, # Restict the experiment to 6 iterations
                          max_concurrent_runs=2) # Run up to 2 iterations in parallel

# 运行试验
experiment = Experiment(workspace=ws, name='mslearn-diabetes-hyperdrive')
run = experiment.submit(config=hyperdrive)

# 显示试验运行时笔记本中的状态
RunDetails(run).show()
run.wait_for_completion()

可在上面的小组件中查看试验运行状态。还可以在 [Azure 机器学习工作室](https://ml.azure.com)中查看主要的 Hyperdrive 试验运行及其子运行。

> **备注**： 如果系统显示了一条表示不能显示非数值内容的消息，可以忽略它。

## 确定性能最佳的运行

完成所有运行后，可以根据指定的性能指标找到最佳运行（在本例中为具有最佳 AUC 的运行）。

In [ ]:
# 打印所有子运行，按主要指标排序
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

# 获取最佳运行及其指标和参数
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',script_arguments)

你已找到最佳运行，现在可以注册它所训练的模型。

In [ ]:
from azureml.core import Model

# 注册模型
best_run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Hyperdrive'},
                        properties={'AUC': best_run_metrics['AUC'], 'Accuracy': best_run_metrics['Accuracy']})

# 列出已注册的模型
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **更多信息**： 有关 Hyperdrive 的详细信息，请参阅 [Azure ML 文档](https://docs.microsoft.com/azure/machine-learning/how-to-tune-hyperparameters)。